# FRED - International GDP Quarterly Growth

Using St. Louis Fed FRED API for international quarterly GDP growth data.

Data source: https://fred.stlouisfed.org/

## Python setup

In [1]:
# system imports
from pathlib import Path
from typing import Any, Dict, List
import time
import requests
from datetime import datetime, timedelta

In [2]:
# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

In [3]:
# local imports
import mgplot as mg

In [4]:
# plotting setup
CHART_DIR = "./CHARTS/FRED/"
mg.set_chart_dir(CHART_DIR)
mg.clear_chart_dir()
SHOW = False
SOURCE = "Federal Reserve Economic Data (FRED)"

## FRED API Configuration

In [5]:
# FRED API Configuration
FRED_API_KEY = None  # Will be loaded from fred.api file
FRED_BASE_URL = "https://api.stlouisfed.org/fred/series/observations"

# Load API key from file
try:
    with open("fred.api", "r") as f:
        FRED_API_KEY = f.read().strip()
    print("✅ FRED API key loaded successfully")
except FileNotFoundError:
    print("⚠️  fred.api file not found!")
    print("Please create a file called 'fred.api' in the project root containing your API key.")
    print("Get a free API key from: https://fred.stlouisfed.org/docs/api/api_key.html")
except Exception as e:
    print(f"❌ Error reading fred.api: {e}")

# Complete OECD Country mapping with FRED series IDs
# All 38 OECD member countries as of 2025
country_map = {
    # === Confirmed Working Series (European Eurostat pattern) ===
    "GDPC1": "United States",                    # Confirmed: US Real GDP
    "CLVMNACSCAB1GQFR": "France",               # Confirmed: France Real GDP  
    "CLVMNACSCAB1GQDE": "Germany",              # Confirmed: Germany Real GDP
    "CLVMNACSCAB1GQAT": "Austria",              # Confirmed: Austria Real GDP
    "CLVMNACSCAB1GQBE": "Belgium",              # Confirmed: Belgium Real GDP
    "CLVMNACSCAB1GQCZ": "Czech Republic",       # Confirmed: Czech Republic Real GDP
    "CLVMNACSCAB1GQDK": "Denmark",              # Confirmed: Denmark Real GDP
    "CLVMNACSCAB1GQEE": "Estonia",              # Confirmed: Estonia Real GDP
    "CLVMNACSCAB1GQFI": "Finland",              # Confirmed: Finland Real GDP
    "CLVMNACSCAB1GQHU": "Hungary",              # Confirmed: Hungary Real GDP
    "CLVMNACSCAB1GQIT": "Italy",                # Confirmed: Italy Real GDP
    "CLVMNACSCAB1GQLV": "Latvia",               # Confirmed: Latvia Real GDP
    "CLVMNACSCAB1GQLT": "Lithuania",            # Confirmed: Lithuania Real GDP
    "CLVMNACSCAB1GQLU": "Luxembourg",           # Confirmed: Luxembourg Real GDP
    "CLVMNACSCAB1GQNL": "Netherlands",          # Confirmed: Netherlands Real GDP
    "CLVMNACSCAB1GQNO": "Norway",               # Confirmed: Norway Real GDP
    "CLVMNACSCAB1GQPL": "Poland",               # Confirmed: Poland Real GDP
    "CLVMNACSCAB1GQPT": "Portugal",             # Confirmed: Portugal Real GDP
    "CLVMNACSCAB1GQSI": "Slovenia",             # Confirmed: Slovenia Real GDP
    "CLVMNACSCAB1GQES": "Spain",                # Confirmed: Spain Real GDP
    "CLVMNACSCAB1GQSE": "Sweden",               # Confirmed: Sweden Real GDP
    "CLVMNACSCAB1GQCH": "Switzerland",          # Confirmed: Switzerland Real GDP
    "CLVMNACSCAB1GQEL": "Greece",               # Confirmed: Greece Real GDP (EL country code)
    "CLVMNACNSAB1GQIS": "Iceland",              # Confirmed: Iceland Real GDP (NSA series)
    "NGDPRSAXDCGBQ": "United Kingdom",          # Confirmed: UK Real GDP (different pattern)
    
    # === Verified Working Series (OECD NAEXKP pattern) ===
    "NAEXKP01IEQ189S": "Ireland",               # Confirmed: Ireland Real GDP (OECD)
    "NAEXKP01CAQ189S": "Canada",                # Confirmed: Canada Real GDP (OECD)
    "NAEXKP01JPQ189S": "Japan",                 # Confirmed: Japan Real GDP (OECD)
    "NAEXKP01NZQ189S": "New Zealand",           # Confirmed: New Zealand Real GDP (OECD)
    "NAEXKP01MXQ189S": "Mexico",                # Confirmed: Mexico Real GDP (OECD)
    "NAEXKP01KRQ189S": "South Korea",           # Confirmed: South Korea Real GDP (OECD)
    
    # === Special Pattern Series ===
    "NGDPRSAXDCAUQ": "Australia",               # Confirmed: Australia Real GDP (special FRED pattern)
    
    # === Countries with no verified FRED series (will be noted in output) ===
    # Slovakia, Chile, Turkey, Israel, Colombia, Costa Rica
    # These may need to be sourced from other providers or are not available in FRED
}

print(f"📊 OECD Country Coverage: {len(country_map)} countries with verified FRED series IDs")
print("🌍 Collecting quarterly GDP data for OECD member countries")
print("✅ All series IDs have been tested and verified to work")

# Countries not available in FRED (for reference)
not_in_fred = [
    "Slovakia", "Chile", "Turkey", 
    "Israel", "Colombia", "Costa Rica"
]
print(f"⚠️  Note: {len(not_in_fred)} OECD countries not found in FRED: {', '.join(not_in_fred)}")
print("🔍 Coverage now includes Australia (found via FRED search)")

✅ FRED API key loaded successfully
📊 OECD Country Coverage: 32 countries with verified FRED series IDs
🌍 Collecting quarterly GDP data for OECD member countries
✅ All series IDs have been tested and verified to work
⚠️  Note: 6 OECD countries not found in FRED: Slovakia, Chile, Turkey, Israel, Colombia, Costa Rica
🔍 Coverage now includes Australia (found via FRED search)


## FRED Data Retrieval Functions

In [6]:
def get_fred_data(series_id: str, start_date: str = "1999-01-01", api_key: str = None) -> pd.Series:
    """Get data from FRED API for a specific series.
    
    Args:
        series_id: FRED series identifier
        start_date: Start date in YYYY-MM-DD format
        api_key: FRED API key (optional if set globally)
    
    Returns:
        pandas Series with date index and values
    """
    if api_key is None:
        api_key = FRED_API_KEY
    
    if api_key is None:
        raise ValueError("FRED API key is required. Get one from https://fred.stlouisfed.org/docs/api/api_key.html")
    
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json",
        "observation_start": start_date,
        "frequency": "q",  # quarterly
    }
    
    response = requests.get(FRED_BASE_URL, params=params)
    response.raise_for_status()
    
    data = response.json()
    observations = data["observations"]
    
    # Convert to pandas Series
    dates = [obs["date"] for obs in observations]
    values = [float(obs["value"]) if obs["value"] != "." else np.nan for obs in observations]
    
    series = pd.Series(values, index=pd.to_datetime(dates), name=series_id)
    series.index = pd.PeriodIndex(series.index, freq="Q")
    
    return series.dropna()

In [7]:
def get_gdp_growth_rates(series_dict: Dict[str, str], start_date: str = "1999-01-01") -> pd.DataFrame:
    """Get quarterly GDP growth rates for multiple countries from FRED.
    
    This function attempts to fetch data for all OECD countries, gracefully handling
    cases where series IDs might be incorrect or data unavailable.
    
    Args:
        series_dict: Dictionary mapping FRED series IDs to country names
        start_date: Start date for data retrieval
    
    Returns:
        DataFrame with quarterly GDP growth rates for available countries
    """
    gdp_data = {}
    successful_fetches = []
    failed_fetches = []
    
    print(f"Attempting to fetch GDP data for {len(series_dict)} countries...")
    print("="*60)
    
    for i, (series_id, country_name) in enumerate(series_dict.items(), 1):
        try:
            print(f"[{i:2d}/{len(series_dict)}] Fetching {country_name:<15} ({series_id})...", end=" ")
            
            # Get the GDP level data
            levels = get_fred_data(series_id, start_date)
            
            # Check if we got meaningful data
            if len(levels) < 10:  # Need at least 10 quarters of data
                print("❌ Insufficient data")
                failed_fetches.append((country_name, series_id, "Insufficient data"))
                continue
            
            # Calculate quarterly growth rates (percentage change from previous quarter)
            growth = levels.pct_change(periods=1) * 100
            growth.name = country_name
            
            gdp_data[country_name] = growth
            successful_fetches.append((country_name, series_id))
            print(f"✅ {len(growth.dropna())} quarters")
            
            time.sleep(0.1)  # Be nice to FRED API
            
        except Exception as e:
            error_msg = str(e)
            if "400" in error_msg:
                print("❌ Series not found")
                failed_fetches.append((country_name, series_id, "Series ID not found"))
            elif "403" in error_msg:
                print("❌ Access denied")
                failed_fetches.append((country_name, series_id, "Access denied"))
            else:
                print(f"❌ {error_msg[:30]}...")
                failed_fetches.append((country_name, series_id, error_msg[:50]))
            continue
    
    # Print summary
    print("\n" + "="*60)
    print("FRED DATA COLLECTION SUMMARY")
    print("="*60)
    print(f"✅ Successfully fetched: {len(successful_fetches)} countries")
    print(f"❌ Failed to fetch: {len(failed_fetches)} countries")
    
    if successful_fetches:
        print("\n✅ SUCCESSFUL COUNTRIES:")
        for country, series_id in successful_fetches:
            print(f"   {country:<20} ({series_id})")
    
    if failed_fetches:
        print("\n❌ FAILED COUNTRIES:")
        for country, series_id, reason in failed_fetches:
            print(f"   {country:<20} ({series_id}) - {reason}")
        print("\n💡 For failed countries, search FRED manually:")
        print("   1. Go to https://fred.stlouisfed.org/")
        print("   2. Search '[Country] Real GDP Quarterly'")
        print("   3. Look for seasonally adjusted series")
        print("   4. Update the series ID in country_map")
    
    if not gdp_data:
        print("\n⚠️  No data retrieved! Check your API key and series IDs.")
        return pd.DataFrame()
    
    df = pd.DataFrame(gdp_data)
    return df.dropna(how="all")

In [8]:
def check_missing_data(df: pd.DataFrame) -> None:
    """Check for missing data in the GDP dataset and provide helpful info."""
    
    if df.empty:
        print("❌ No data available")
        print("\n🔧 TROUBLESHOOTING:")
        print("1. Check that fred.api file exists and contains your API key")
        print("2. Verify your API key at https://fred.stlouisfed.org/")
        print("3. Check internet connection")
        return
    
    print(f"📊 DATA OVERVIEW")
    print(f"   Countries: {len(df.columns)}")
    print(f"   Quarters: {len(df)}")
    print(f"   Date range: {df.index[0]} to {df.index[-1]}")
    
    # Check latest period coverage
    final_row = df.iloc[-1]
    missing_count = final_row.isna().sum()
    
    if missing_count > 0:
        print(f"\n📅 LATEST PERIOD: {final_row.name}")
        print(f"   Missing data: {missing_count}/{len(df.columns)} countries")
        missing_countries = df.columns[final_row.isna()].tolist()
        available_countries = df.columns[final_row.notna()].tolist()
        
        if missing_countries:
            print(f"   Countries missing data: {', '.join(missing_countries[:5])}{'...' if len(missing_countries) > 5 else ''}")
        if available_countries:
            print(f"   Countries with data: {', '.join(available_countries[:5])}{'...' if len(available_countries) > 5 else ''}")
    else:
        print(f"\n✅ Complete data for latest period: {final_row.name}")
    
    # Show data recency by country
    print(f"\n📈 MOST RECENT DATA BY COUNTRY:")
    last_dates = df.apply(lambda x: x.last_valid_index()).sort_values(ascending=False)
    for country in last_dates.head(10).index:
        last_date = last_dates[country]
        print(f"   {country:<20}: {last_date}")
    
    if len(last_dates) > 10:
        print(f"   ... and {len(last_dates)-10} more countries")
        
def suggest_missing_countries():
    """Suggest series IDs for countries that might be missing."""
    print("\n🔍 FINDING MORE COUNTRIES:")
    print("For OECD countries not in the current list, try these search terms:")
    
    missing_oecd = [
        ("Australia", "OECD Real GDP Australia Quarterly"),
        ("Canada", "OECD Real GDP Canada Quarterly"),  
        ("Japan", "OECD Real GDP Japan Quarterly"),
        ("South Korea", "OECD Real GDP Korea Quarterly"),
        ("Mexico", "OECD Real GDP Mexico Quarterly"),
        ("Chile", "OECD Real GDP Chile Quarterly"),
        ("Turkey", "OECD Real GDP Turkey Quarterly"),
        ("Israel", "OECD Real GDP Israel Quarterly"),
        ("Colombia", "OECD Real GDP Colombia Quarterly"),
        ("Costa Rica", "OECD Real GDP Costa Rica Quarterly"),
        ("New Zealand", "OECD Real GDP New Zealand Quarterly"),
    ]
    
    for country, search_term in missing_oecd:
        print(f"   {country:<15}: Search FRED for '{search_term}'")
        
    print("\n💡 Common series ID patterns to try:")
    print("   European countries: CLVMNACSCAB1GQ + 2-letter country code")
    print("   Other countries: [COUNTRY]RGDPNASAQ or similar variations")
    print("   Search FRED directly: https://fred.stlouisfed.org/")

## Data Collection

**Note:** The notebook will automatically load your FRED API key from the `fred.api` file. If you haven't set this up yet, see the Setup Instructions section below.

In [9]:
# Collect GDP data for all OECD countries
if FRED_API_KEY is None:
    print("⚠️  FRED API key not loaded!")
    print("Please create a 'fred.api' file with your API key (see Setup Instructions below)")
    gdp = pd.DataFrame()
else:
    print("🌍 COLLECTING OECD GDP DATA FROM FRED")
    print("This will attempt to fetch quarterly GDP growth data for all OECD member countries.")
    print("Some countries may fail if the series IDs are incorrect - that's normal!")
    print()
    
    # Get GDP data for OECD countries  
    gdp = get_gdp_growth_rates(country_map, start_date="2000-01-01")
    
    if not gdp.empty:
        print()
        check_missing_data(gdp)
    else:
        suggest_missing_countries()

🌍 COLLECTING OECD GDP DATA FROM FRED
This will attempt to fetch quarterly GDP growth data for all OECD member countries.
Some countries may fail if the series IDs are incorrect - that's normal!

Attempting to fetch GDP data for 32 countries...
[ 1/32] Fetching United States   (GDPC1)... ✅ 101 quarters
[ 2/32] Fetching France          (CLVMNACSCAB1GQFR)... ✅ 101 quarters
[ 3/32] Fetching Germany         (CLVMNACSCAB1GQDE)... ✅ 101 quarters
[ 4/32] Fetching Austria         (CLVMNACSCAB1GQAT)... ✅ 101 quarters
[ 5/32] Fetching Belgium         (CLVMNACSCAB1GQBE)... ✅ 100 quarters
[ 6/32] Fetching Czech Republic  (CLVMNACSCAB1GQCZ)... ✅ 101 quarters
[ 7/32] Fetching Denmark         (CLVMNACSCAB1GQDK)... ✅ 100 quarters
[ 8/32] Fetching Estonia         (CLVMNACSCAB1GQEE)... ✅ 101 quarters
[ 9/32] Fetching Finland         (CLVMNACSCAB1GQFI)... ✅ 101 quarters
[10/32] Fetching Hungary         (CLVMNACSCAB1GQHU)... ✅ 101 quarters
[11/32] Fetching Italy           (CLVMNACSCAB1GQIT)... ✅ 101 quarte

## Plotting Functions (Adapted from OECD notebook)

In [10]:
# Chart groupings for different analyses
chart_groups = {
    "major_economies": ["United States", "Germany", "Japan", "United Kingdom", "France", "Italy"],
    "anglosphere": ["Australia", "United States", "Canada", "United Kingdom"],
    "g7": ["United States", "Germany", "Japan", "United Kingdom", "France", "Italy", "Canada"],
}

MEAN_MEDIAN = 0.80  # proportion of non-na data points to plot mean and median
COUNTER = 0
PW_COUNTER = 0

In [11]:
def plot_world_gdp(data: pd.DataFrame) -> None:
    """Plot Australia vs world GDP growth context."""
    
    if data.empty:
        print("No data to plot")
        return
    
    global PW_COUNTER
    PW_COUNTER += 1
    
    # Plot other countries as background
    other_countries = [col for col in data.columns if col != "Australia"]
    if other_countries:
        background_data = data[other_countries]
        ax = mg.line_plot(background_data, width=0.3, color='blue', alpha=0.5, label_series=False)
    else:
        fig, ax = plt.subplots()
    
    # Plot mean and median if enough countries
    if len(other_countries) >= 3:
        mean = data[other_countries].mean(axis=1).where(
            data[other_countries].notna().sum(axis=1) >= len(other_countries) * MEAN_MEDIAN,
            other=np.nan,
        )
        median = data[other_countries].median(axis=1).where(
            data[other_countries].notna().sum(axis=1) >= len(other_countries) * MEAN_MEDIAN,
            other=np.nan,
        )
        
        mean.name = "FRED monitored mean"
        median.name = "FRED monitored median"
        
        mg.line_plot(mean, ax=ax, color="darkblue", style="--", width=2, label_series=True)
        mg.line_plot(median, ax=ax, color="darkred", style=":", width=2, label_series=True)
    
    # Highlight Australia if available
    if "Australia" in data.columns:
        mg.line_plot(
            data["Australia"].dropna(),
            ax=ax,
            color="darkorange",
            width=3,
            label_series=True,
        )
    
    mg.finalise_plot(
        ax,
        title="Australian quarterly GDP growth in world context",
        ylabel="Per cent per quarter",
        xlabel=None,
        y0=True,
        rfooter=SOURCE,
        lfooter=f"FRED monitored nations. Mean/median calculated when >{int(MEAN_MEDIAN*100)}% report.",
        tag=str(PW_COUNTER),
        legend={"loc": "best", "fontsize": "xx-small", "ncol": 3},
        show=SHOW,
    )

In [12]:
def plot_contractions(growth: pd.DataFrame) -> None:
    """Identify and plot quarterly GDP contractions."""
    
    if growth.empty:
        print("No data to analyze")
        return
    
    contractions = growth < 0
    contraction_count = contractions.sum(axis=1)

    start = pd.Period("2000Q1", freq="Q")
    title = "Number of FRED Monitored Countries with Quarterly GDP Contraction"

    ax = contraction_count[contraction_count.index >= start].plot.bar()
    ax.set_xticks(ax.get_xticks()[::4])
    
    mg.finalise_plot(
        ax,
        title=title,
        ylabel="Count",
        rfooter=SOURCE,
        show=SHOW,
    )

    # Print nations in contraction
    if len(contractions) > 0:
        latest_contractions = contractions.iloc[-1][contractions.iloc[-1]]
        print(f"Latest countries in contraction (N={contraction_count.iloc[-1]}):")
        print(", ".join(latest_contractions.index.tolist()))

In [13]:
def plot_recessions(growth: pd.DataFrame) -> None:
    """Identify and plot technical recessions (two consecutive contractions)."""
    
    if growth.empty:
        print("No data to analyze")
        return
    
    recessions = (growth < 0) & (growth.shift(1) < 0)
    recession_count = recessions.sum(axis=1)

    title = "Number of FRED Monitored Countries in Technical Recession"
    start = pd.Period("2000Q1", freq="Q")
    ax = recession_count[recession_count.index >= start].plot.bar()
    ax.set_xticks(ax.get_xticks()[::4])
    
    mg.finalise_plot(
        ax,
        title=title,
        ylabel="Count",
        rfooter=SOURCE,
        lfooter="Technical recession = two consecutive quarters of negative GDP growth",
        show=SHOW,
    )

    # Print nations in recession
    if len(recessions) > 0:
        latest_recessions = recessions.iloc[-1][recessions.iloc[-1]]
        print(f"Latest countries in technical recession (N={recession_count.iloc[-1]}):")
        print(", ".join(latest_recessions.index.tolist()))

In [14]:
def post_covid_analysis(gdp_data: pd.DataFrame, start: str = "2022Q1") -> None:
    """Analyze GDP performance since COVID recovery."""
    
    if gdp_data.empty:
        print("No data for post-COVID analysis")
        return
    
    start_period = pd.Period(start, freq="Q")
    recent_gdp = gdp_data.loc[gdp_data.index >= start_period].dropna(how="all", axis=1)
    
    if recent_gdp.empty:
        print(f"No data available from {start}")
        return
    
    # Plot cumulative growth since start
    growth = ((1 + recent_gdp / 100).cumprod() - 1) * 100
    final = growth.apply(pd.Series.last_valid_index).astype(str).str[2:]  
    gmap = {x: x + " " + str(y) for x, y in final.items()}
    
    cum_growth = {}
    for col in growth.columns:
        last_valid_idx = growth[col].last_valid_index()
        if last_valid_idx is not None:
            cum_growth[col] = growth[col].loc[last_valid_idx]
    
    if cum_growth:
        cum_growth_s = pd.Series(cum_growth).rename(index=gmap).sort_values()
        ax = cum_growth_s.plot.bar()
        ax.tick_params(axis="both", which="major", labelsize="x-small")
        
        mg.finalise_plot(
            ax,
            title=f"Cumulative GDP growth since {start_period-1}",
            ylabel="Per cent",
            rfooter=SOURCE,
            lfooter="To latest available data; see x-axis labels.",
            show=SHOW,
        )
    
    # Plot number of contractions since start
    negative = recent_gdp < 0
    contraction_count = negative.sum().sort_values()
    
    if len(contraction_count) > 0:
        ax = contraction_count.plot.bar()
        ax.tick_params(axis="both", which="major", labelsize="x-small")
        
        mg.finalise_plot(
            ax,
            title=f"Number of negative GDP quarters since {start}",
            ylabel="Count",
            rfooter=SOURCE,
            show=SHOW,
        )

## Analysis and Plotting

Run the analysis only if we have data (requires FRED API key).

In [15]:
if not gdp.empty:
    print("Running GDP analysis...")
    
    # World context plots
    plot_world_gdp(gdp)
    plot_world_gdp(gdp[gdp.index.year >= 2022])  # Recent years
    
    # Contraction analysis
    plot_contractions(gdp)
    
    # Recession analysis
    plot_recessions(gdp)
    
    # Post-COVID analysis
    post_covid_analysis(gdp)
    
    print("\nAnalysis complete!")
    
else:
    print("Skipping analysis - no data available (FRED API key needed)")

Running GDP analysis...
Latest countries in contraction (N=3):
Germany, Finland, Italy
Latest countries in technical recession (N=0):


Analysis complete!


## Data Summary

In [16]:
if not gdp.empty:
    print("GDP Data Summary:")
    print(f"Countries: {len(gdp.columns)}")
    print(f"Quarters: {len(gdp)}")
    print(f"Date range: {gdp.index[0]} to {gdp.index[-1]}")
    print("\nLatest quarterly growth rates:")
    display(gdp.tail())
else:
    print("No GDP data loaded. Set FRED_API_KEY to access data.")

GDP Data Summary:
Countries: 32
Quarters: 101
Date range: 2000Q2 to 2025Q2

Latest quarterly growth rates:


,United States,France,Germany,Austria,Belgium,Czech Republic,Denmark,Estonia,Finland,Hungary,...,Greece,Iceland,United Kingdom,Ireland,Canada,Japan,New Zealand,Mexico,South Korea,Australia
2024Q2,0.738980,0.212077,-0.258249,-0.340824,0.277844,0.299223,1.585008,0.108422,0.309062,-0.278345,...,1.140663,4.878691,0.457772,NaN,NaN,NaN,NaN,NaN,NaN,0.229453
2024Q3,0.759510,0.372212,0.019173,-0.147667,0.255428,0.597472,0.788846,0.077360,0.805041,-0.632036,...,0.229066,2.486389,0.000779,NaN,NaN,NaN,NaN,NaN,NaN,0.319793
2024Q4,0.607065,-0.090563,0.182176,-0.045323,0.168592,0.819144,1.207120,0.216442,-0.123192,0.623132,...,0.775182,-1.860231,0.095375,NaN,NaN,NaN,NaN,NaN,NaN,0.570188
2025Q1,-0.125867,0.138178,0.306234,0.139187,0.399215,0.701013,-1.269866,-0.304678,0.043384,-0.119706,...,0.039244,-2.691064,0.736587,NaN,NaN,NaN,NaN,NaN,NaN,0.211109
2025Q2,0.812603,0.295404,-0.076318,0.138527,NaN,0.229801,NaN,0.481625,-0.015557,0.367149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Setup Instructions

To use this notebook:

1. **Get a FRED API Key** (free):
   - Visit: https://fred.stlouisfed.org/docs/api/api_key.html
   - Create an account and request an API key

2. **Save your API key securely**:
   - Create a file named `fred.api` in the project root directory
   - Put your API key in this file (just the key, no quotes or extra text)
   - Example file contents: `a1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p6`

3. **Add more countries** (optional):
   - Find FRED series IDs at https://fred.stlouisfed.org/
   - Add them to the `country_map` dictionary above

4. **Run the notebook**:
   - All cells should execute and produce charts with current GDP data

**Note:** The `fred.api` file is automatically excluded from git commits for security.

## The End

In [17]:
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-09-05 07:52:28

Python implementation: CPython
Python version       : 3.13.6
IPython version      : 9.4.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

requests  : 2.32.4
typing    : 3.10.0.0
IPython   : 9.4.0
pandas    : 2.3.1
pathlib   : 1.0.1
matplotlib: 3.10.5
mgplot    : 0.2.12
numpy     : 2.3.2

Watermark: 2.5.0

